# 太陽位置の計算と日射量計算プログラム

### 太陽定数

In [1]:
const JSTD = 1367.0

1367.0

## 1. 日射量計算の基本式

全日射は直達日射と天空（散乱）日射に分けることができる。  

### 1.1 Bouguerの式

$$
    J_{dir} = J_{STD} P ^ {\frac{1}{sinH}}
$$

- SINH:太陽高度のサイン
- P:大気透過率
- SOLDN:法線面直達日射量（Bouguer の式）
- SOLSN:法線面天空日射量（Berlage の式）

In [2]:
function cal_soldn( P::Float64, SINH::Float64 )
    return JSTD * P ^ ( 1.0 / SINH )
end
cal_soldn(; P::Float64, SINH::Float64 ) = cal_soldn( P, SINH )

cal_soldn (generic function with 2 methods)

In [3]:
cal_soldn( P = 0.8, SINH = 0.6 )

942.4359108075473

### 1.2 Berlageの式

$$
    J_{dif} = \frac{1}{2}J_{STD}sinH\frac{1-P^{\frac{1}{sinH}}}{1-1.4log_eP}
$$

In [4]:
function cal_solsn( P::Float64, SINH::Float64 )
    return 1.0 / 2.0 * JSTD * SINH * ( 1.0 - P ^ ( 1.0 / SINH ) ) / ( 1.0 - 1.4 *  log(P) )
end
cal_solsn(; P::Float64, SINH::Float64 ) = cal_solsn( P, SINH )

cal_solsn (generic function with 2 methods)

In [5]:
cal_solsn( P = 0.8, SINH = 0.6 )

97.05054285290045

## 2. 太陽位置の計算プログラム

$$ 
    \sin H = \sin \phi \sin \delta + \cos \phi \cos \delta \cos t
$$

$$ 
    \sin A = \frac { \cos \delta \sin t } { \cos H }
$$

$$ 
    \cos A = \frac { \sin h \sin \delta - \sin \delta } { \cos h \cos \phi }
$$

$$
    t = 15(T_m - 12) + (L - L_0) + E_t
$$

- PHI:計算対象地点の緯度（度）
- LON:計算対象地点の経度（度）
- TM:標準時（時）
- LONS:標準時の地点の経度（度） = 135.0
- SINH:太陽高度のサイン
- COSH:太陽高度のコサイン
- SINDLT:赤緯のサイン
- COSDLT:赤緯のコサイン
- ET:均時差（度）（時角）1°が4分
- T:時角（度）

In [6]:
function SUNLHA( PHI::Float64, LON::Float64, TM::Float64, LONS::Float64, SINDLT::Float64, COSDLT::Float64, ET::Float64 )
    #RAD = 2.0 * PI / 360.0

    T = 15.0 * ( TM - 12.0 ) + ( LON - LONS ) + ET
    
    PHIRAD = PHI # * RAD
    TRAD   = T   # * RAD
    
    SINH   = sind(PHIRAD) * SINDLT + cosd(PHIRAD) * COSDLT * cosd(TRAD)
    COSH   = ( 1.0 - SINH ^ 2.0) ^ (1/2)
    SINA   = COSDLT * sind( TRAD ) / COSH
    COSA   = ( SINH * sind( PHIRAD ) - SINDLT ) / ( COSH * cosd( PHIRAD ) )
    
    return SINH, COSH, SINA, COSA
end
SUNLHA(; PHI::Float64, LON::Float64, TM::Float64, LONS::Float64, SINDLT::Float64, COSDLT::Float64, ET::Float64 ) = SUNLHA( PHI, LON, TM, LONS, SINDLT, COSDLT, ET )

SUNLHA (generic function with 2 methods)

### 赤緯と均時差の計算プログラム

$$
    \sin \delta = \cos ( \nu + \epsilon ) \sin \delta _0
$$

- YEAR:西暦
- NDAY:年間通し日
- SINDLT:赤緯のサイン
- COSDLT:赤緯のコサイン
- ET:均時差（度）（時角）1°が4分

In [7]:
function SUNLD( YEAR::Int, NDAY::Int )

    RAD = 2.0 * pi / 360.0
    DLTO= -23.43930 * RAD
    
    N = YEAR - 1968.0

    DO1 = 3.710 + 0.25960 * N - floor( ( N + 3.0 ) / 4.0 )
    
    M = 0.98560 * ( NDAY - DO1 )
    
    EPS = 12.39010 + 0.0172 * ( N + M / 360.0 )
    V   = M + 1.9140 * sind( M ) + 0.02 * sind( 2.0 * M )
    VEPS= ( V + EPS ) * RAD
    
    ET = (M - V) - atan( 0.0430 * sin( 2.0 * VEPS ) / ( 1.0 - 0.0430 * cos( 2.0 * VEPS ) ) ) / RAD
    
    SINDLT = cos(VEPS) * sin(DLTO)
    COSDLT = ( abs( 1.0- SINDLT ^ 2.0 ) )^( 1/2 )
    
    return SINDLT, COSDLT, ET
end
SUNLD(; YEAR::Int, NDAY::Int ) = SUNLD( YEAR, NDAY )

SUNLD (generic function with 2 methods)

### 2.2 太陽高度の計算

In [8]:
function cal_ALTI(SINH::Float64)
    return asin(SINH) * 180 / pi
end
cal_ALTI(;SINH::Float64) = cal_ALTI(SINH)

cal_ALTI (generic function with 2 methods)

### 2.3 太陽方位角の計算

In [9]:
function cal_LATI(COSA::Float64, SINA::Float64)
    if SINA >= 0.0
        LATI = 90.0 - atan( COSA / SINA ) * 180 / pi
    elseif SINA <= 0.0
        LATI = -90.0 - atan( COSA / SINA ) * 180 / pi
    end
    return LATI
end
cal_LATI(;COSA::Float64, SINA::Float64) = cal_LATI(COSA, SINA)

cal_LATI (generic function with 2 methods)

#### Appendix. NDAY（積算日数）の計算

In [10]:
function cal_NDAY( year::Int, month::Int, day::Int )
    if month == 1;      NDAY = day
    elseif month == 2;  NDAY = 31 + day
    elseif month == 3;  NDAY = 59 + day
    elseif month == 4;  NDAY = 90 + day
    elseif month == 5;  NDAY =120 + day
    elseif month == 6;  NDAY =151 + day
    elseif month == 7;  NDAY =181 + day
    elseif month == 8;  NDAY =212 + day
    elseif month == 9;  NDAY =243 + day
    elseif month == 10; NDAY =273 + day
    elseif month == 11; NDAY =304 + day
    elseif month == 12; NDAY =334 + day
    end
    
    # うるう年の計算
    if ( ( mod(year,4)==0 && mod(year,100)≠0) || ( mod(year,400)==0 ) ) && ( month > 2 )
        NDAY = NDAY + 1
    end
    
    return NDAY
end
cal_NDAY(; year::Int, month::Int, day::Int ) = cal_NDAY( year, month, day )

cal_NDAY (generic function with 2 methods)

### 計算の確認

In [11]:
SINDLT, COSDLT, ET = SUNLD( YEAR = 2022, NDAY = 5 )
LON  = 135.678 # 桂キャンパスの経度
PHI  = 34.983  # 桂キャンパスの緯度
LONS = 135.0
TM = 12.0
SINH, COSH, SINA, COSA = SUNLHA( PHI, LON, TM, LONS, SINDLT, COSDLT, ET )
println( asind(SINH) )
println( acosd(COSA) )

32.37611354311369
0.6715704788474401


## 3. 壁面に入射する日射量の計算  

参考：エース建築環境工学I pp.29-30|

### 3.1 ある時刻における法線面日射量・天空日射量、太陽高度・方位角の算出


- SOLDN；法線面直達日射量（Bouguer の式より算出）
- SOLSN：法線面天空日射量（Berlage の式より算出）

In [12]:
function cal_solar_radiation( year::Int, month::Int, day::Int, hour::Int, min::Int, sec::Int, 
    PHI::Float64, LON::Float64, LONS::Float64, P::Float64 )

    NDAY  = cal_NDAY( year, month, day )
    
    # 地方標準時
    TM = Float64(hour) + Float64(min) / 60.0 + Float64(sec) / 3600.0
    
    # 赤緯の計算
    SINDLT, COSDLT, ET = SUNLD( YEAR = year, NDAY = NDAY )
    
    # 太陽高度・太陽方位角の計算
    SINH, COSH, SINA, COSA = SUNLHA( PHI, LON, TM, LONS, SINDLT, COSDLT, ET )

    # 2024/11/28追記：壁面日射量が合わなかったため修正
    #LATI = 180.0 - acos(COSA) * 180.0 / pi
    if SINA >= 0.0
        LATI = 90.0 - atan(COSA/SINA) * 180.0 / pi
    elseif SINA < 0.0
        LATI = - 90.0 - atan(COSA/SINA) * 180.0 / pi
    end
    ALTI = asin(SINH) * 180.0 / pi
    
    # 日射量の計算
    # 太陽高度が0以上のとき
    if SINH > 0
        SOLDN = cal_soldn( P, SINH )
        SOLSN = cal_solsn( P, SINH )
    # 太陽が沈んでいるとき
    else
        SOLDN = 0.0
        SOLSN = 0.0
    end
    
    return SOLDN, SOLSN, LATI, ALTI
end
cal_solar_radiation(; year::Int, month::Int, day::Int, hour::Int, min::Int, sec::Int, PHI::Float64, LON::Float64, LONS::Float64, P::Float64) = cal_solar_radiation( year, month, day, hour, min, sec, PHI, LON, LONS, P)

cal_solar_radiation (generic function with 2 methods)

### 3.2 壁面に入射する日射量

#### 3.2.1 壁面（傾斜面）への直達日射量  

$$ 
    J_{di} = J_{STD_d} cos(i) 
$$

##### 傾斜面への太陽光線の入射角

$$ cos(i) = cos(\theta_e)sin(h)+sin(\theta_e)cos(h)cos(\alpha_s - \theta_a) $$

- RADDN：直達日射量
- RADSN：天空放射量
- $\theta_a$：傾斜面の方位角
- $\theta_e$：傾斜角  

In [13]:
function cal_direct_solar_radiation_at_wall( SOLDN::Float64, ALTI::Float64, LATI::Float64, θe::Float64, θa::Float64,
    LATI_started::Float64 = -180.0, LATI_limited::Float64 = 360.0, ALTI_started::Float64 = -180.0, ALTI_limited::Float64 = 360.0 )
    RAD = 180 / pi
    COSI = cos( θe / RAD ) * sin( ALTI / RAD ) + sin( θe / RAD ) * cos( ALTI / RAD ) * cos( ( LATI - θa ) / RAD )

    if (COSI >= 0.0)
        # 直達日射の遮蔽がない場合
        if LATI >= LATI_started && LATI <= LATI_limited && ALTI >= ALTI_started && ALTI <= ALTI_limited
            RADDN = SOLDN * COSI
        # 直達日射の遮蔽がある場合
        else
            RADDN = 0.0
        end
    else
        RADDN = 0.0
    end    

    return RADDN
end

cal_direct_solar_radiation_at_wall(; SOLDN::Float64, ALTI::Float64, LATI::Float64, θe::Float64, θa::Float64,
    LATI_started::Float64 = -180.0, LATI_limited::Float64 = 360.0, ALTI_started::Float64 = -180.0, ALTI_limited::Float64 = 360.0 ) = 
        cal_direct_solar_radiation_at_wall( SOLDN, ALTI, LATI, θe, θa, LATI_started, LATI_limited, ALTI_started, ALTI_limited )

cal_direct_solar_radiation_at_wall (generic function with 6 methods)

In [14]:
cal_direct_solar_radiation_at_wall( SOLDN = 100.0, ALTI = 10.0, LATI = 10.0, θe = 0.0, θa = 0.0 )

17.364817766693033

#### 3.2.2 壁面（傾斜面）への天空日射量  

$$ 
    J_{si} = J_{H} \Bigl( \frac{ 1 + \cos \theta_e }{ 2 } \Bigl )
$$

In [15]:
function cal_diffused_solar_radiation_at_wall( SOLSN::Float64, θe::Float64 )
    RAD = 180 / pi
    # RADSN = SOLSN * ( abs( ( 180.0 - γ ) / 180.0 ) )
    RADSN = SOLSN * ( ( 1.0 + cos( θe / RAD ) ) / 2.0 )
    return RADSN
end
cal_diffused_solar_radiation_at_wall(; SOLSN::Float64, θe::Float64 ) = cal_diffused_solar_radiation_at_wall( SOLSN, θe )

cal_diffused_solar_radiation_at_wall (generic function with 2 methods)

### 3.2.3 地面からの反射日射
地面からの反射日射は、地面が均等拡散面であると仮定し以下のように与える。
$$
    J_r = \rho J_H \Bigl( \frac{  1 - \cos \theta_e }{ 2 } \Bigl )
$$

In [16]:
function cal_reflected_solar_radiation_by_ground( rho::Float64, SOLDN::Float64, SOLSN::Float64, θe::Float64 )
    return rho * ( SOLSN + SOLDN ) * ( ( 1.0 - cos( θe / (180 / pi) ) ) / 2.0 )
end
cal_reflected_solar_radiation_by_ground(; SOLDN::Float64, SOLSN::Float64, θe::Float64, rho::Float64 = 0.2 ) = cal_reflected_solar_radiation_by_ground( rho, SOLDN, SOLSN, θe )

cal_reflected_solar_radiation_by_ground (generic function with 2 methods)

## 4. 夜間放射の計算式

### 4.1 放射（輻射）の基礎式

物体からの放射熱量は一般に以下のように表される。
$$
    Q_r = \varepsilon \sigma T^{4}
$$

- $T$：温度[K]
- $\varepsilon$：物質の長波放射率[-]
- $\sigma$：ステファンボルツマン定数[W/m<sup>-2</sup>K<sup>4</sup>]

In [17]:
function cal_flux_thermal_radiation( temp::Float64, ε::Float64 )
    return ε * temp ^ 4 * 5.67e-8
end

cal_flux_thermal_radiation (generic function with 1 method)

### 4.2 大気放射

Bruntの式は、地表面からの長波放射とそれに対する大気の放射効果（大気放射）を評価するために用いられる経験式の一つである。  
参考：高橋尚人ら, "曇天時における大気放射量推定に関する研究", 雪氷研究大会（2013）  

Bruntは水蒸気圧に応じた大気の放射率の表現として以下の形を提案している。
$$
    \varepsilon_a = a + b \sqrt{f}
$$

- $f$：水蒸気圧[Pa]  

なお、a, bは経験定数であり地域が条件により異なる。  
例えば、山本・Bruntの式の場合、大気放射は以下のように与えられる。

$$
    Q_{ra} = \sigma T_{a}^{4} ( 0.51 + 0.0066 \sqrt{f} )
$$



In [18]:
function cal_flux_thermal_radiation_in_atmospher_cloudless( temp::Float64, f::Float64 )
    return 5.67e-8 * temp ^4 * ( 0.51 + 0.0066 * f^(1/2) ) 
end

cal_flux_thermal_radiation_in_atmospher_cloudless (generic function with 1 method)

ただし、Bruntの式は晴天時を対象とした式である。  
そのため、曇天時長波放射量推定式がいくつか提案されており、ここでは以下に示すSellersの式を用いる。

$$
    Q_{ra}^{'} = 0.75 n^2 \sigma T_{a}^{4} + Q_{ra} ( 1 - 0.75 n^2 )
$$

- $n$：（下層）雲量  

※ただし、雲量は通常0~10の値を取るため、最大で1となるよう補正した。

In [19]:
function cal_flux_thermal_radiation_in_atmospher_cloudy( temp::Float64, f::Float64, n::Int )
    return 0.75 * (n/10)^2 * 5.67e-8 * temp ^4 + ( 1.0 - 0.75 * (n/10)^2 ) * cal_flux_thermal_radiation_in_atmospher_cloudless( temp, f )
end

cal_flux_thermal_radiation_in_atmospher_cloudy (generic function with 1 method)

#### 4.3 傾斜角を考慮した実効放射

傾斜面の場合、天空率が変化するため実効放射には形態係数を乗じる必要がある。  
周辺地盤および建物の表面温度が計算対象の壁面表面温度と同一であると仮定すれば、傾斜角$\beta$の面の実効放射以下のように与えられる。  

$$
    J_{er} = \phi J_{er}^{'} = \Bigl( \frac{ 1 + \cos \theta_e }{ 2 } \Bigl ) J_{er}^{'}
$$

なお、$\phi$は形態係数である。  

また壁面に入射する正味の熱放射は与式より以下のように与えられる。  

$$
    J_{er}^{'} = Q_{ra}^{'} - Q_r
$$

従って、壁面における実効放射は以下のように与えられる。

$$
    J_{er} = \Bigl( \frac{ 1 + \cos \theta_e }{ 2 } \Bigl ) ( Q_{ra}^{'} - Q_r )
$$

In [20]:
function cal_effective_thermal_radiation_at_wall( temp_wall::Float64, temp_air::Float64, ε::Float64, pv::Float64, n::Int, θe::Float64 )
    Qr      = cal_flux_thermal_radiation( temp_wall, ε )
    Qrad    = cal_flux_thermal_radiation_in_atmospher_cloudy( temp_air, pv, n )
    return ( ( 1.0 + cos( θe / (180 / pi) ) ) / 2.0 ) * ( Qrad - Qr )
end

cal_effective_thermal_radiation_at_wall(; temp_wall::Float64, temp_air::Float64, ε::Float64, pv::Float64, n::Int, θe::Float64 ) = cal_effective_thermal_radiation_at_wall( temp_wall, temp_air, ε, pv, n, θe )

cal_effective_thermal_radiation_at_wall (generic function with 2 methods)